In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (without a graphical user interface)
chrome_service = ChromeService(executable_path='/Users/samdvorin/Desktop/code/542/weather-prediction/chromedriver')  # Replace with the path to your chromedriver executable
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

url = 'https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/2024-3-1'
driver.get(url)

# Locate the table element at the bottom of the page using your specified XPath
table_element = driver.find_element('xpath', '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]')

# Extract the text from the table
table_text = table_element.text
print("Table at the bottom of the page:\n", table_text)

driver.quit()


In [ ]:
import os
copy_text = table_text

with open(os.path.join("sampledata.txt"), 'w') as file:
    file.write(copy_text)
print(copy_text)

In [ ]:
# doesnt take no wind speed into account
import pandas as pd

# Read the text file and parse the data
data = []
with open('sampledata.txt', 'r') as file:
    for line in file:
        # Split the line by spaces
        parts = line.strip().split()

        # Ensure that the line has the expected number of elements
        if len(parts) >= 10:
            # Extract the time, AM/PM, temperature, dew point, humidity, and wind speed
            time = parts[0]
            am_pm = parts[1]
            temp = int(parts[2])
            dew_point = int(parts[4])
            humidity = int(parts[6])
            wind_speed = parts[9]

            # Append the parsed data to the list
            data.append([time, am_pm, temp, dew_point, humidity, wind_speed])

# Create a DataFrame from the list
df = pd.DataFrame(data, columns=['time', 'am/pm', 'temp', 'dew point', 'humidity', 'wind speed'])

# Display the DataFrame
print(df.head())


   time am/pm  temp  dew point  humidity wind speed
0  1:51    AM    33         11        40         13
1  2:51    AM    32         11        42         13
2  3:51    AM    32         10        40          9
3  4:51    AM    31         11        43         13
4  5:51    AM    31         11        43         10


In [11]:
import pandas as pd

# Read the text file and parse the data
data = []
with open('sampledata.txt', 'r') as file:
    for line in file:
        # Split the line by spaces
        parts = line.strip().split()

        # Ensure that the line has the expected number of elements
        if len(parts) >= 10:
            # Extract the time, AM/PM, temperature, dew point, humidity, and wind speed
            time = parts[0]
            am_pm = parts[1]
            temp = int(parts[2])
            dew_point = int(parts[4])
            humidity = int(parts[6])
            # Check if wind speed is available
            wind_speed_index = 9
            if parts[wind_speed_index].isdigit():
                wind_speed = int(parts[wind_speed_index])
            else:
                wind_speed = 0

            # Append the parsed data to the list
            data.append([time, am_pm, temp, dew_point, humidity, wind_speed])

# Create a DataFrame from the list
df = pd.DataFrame(data, columns=['time', 'am/pm', 'temp', 'dew point', 'humidity', 'wind speed'])

# Display the DataFrame
print(df.head(24))


     time am/pm  temp  dew point  humidity  wind speed
0    1:51    AM    33         11        40          13
1    2:51    AM    32         11        42          13
2    3:51    AM    32         10        40           9
3    4:51    AM    31         11        43          13
4    5:51    AM    31         11        43          10
5    6:51    AM    31         11        43           6
6    7:51    AM    31         12        46           6
7    8:51    AM    33         12        42           5
8    9:51    AM    35         13        40           8
9   10:51    AM    38         14        38           0
10  11:51    AM    40         14        35           7
11  12:51    PM    42         13        31           6
12   1:51    PM    43         16        34          10
13   2:51    PM    43         17        35          14
14   3:51    PM    42         19        40          18
15   4:51    PM    42         20        41          16
16   5:51    PM    41         22        47          13
17   6:51 

In [ ]:
# OUTDATED first try at converting to dataframe, ineffeicnt and converts to csv then back to dataframe


import csv
import pandas as pd
#print(table_text)

# Split the table text into rows
rows = [row.split() for row in copy_text.strip().split('\n')[10:]]
# Specify the CSV file path
csv_file_path = 'table_data.csv'

# Write data to CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    csv_writer.writerows(rows)

print(f"Data has been written to {csv_file_path}")


# Read the CSV file into a list of lines
with open(csv_file_path, 'r') as file:
    lines = file.readlines()

# Modify each line to exclude the last value if it is the 19th value
modified_lines = [','.join(line.split(',')[:-1]) + '\n' if len(line.split(',')) == 19 else line for line in lines]

# Save the modified content back to the CSV file
with open(csv_file_path, 'w') as modified_file:
    modified_file.writelines(modified_lines)


# Read the CSV file into a DataFrame and drop the last column
df = pd.read_csv(csv_file_path)

# Display the DataFrame
#print(df.head())

df = df.iloc[:, [0, 1, 2, 4, 6, 9]]
column_names = df.columns.tolist()

# Create a new DataFrame with the column names as a row
column_names_row = pd.DataFrame([column_names], columns=df.columns)

# Concatenate the new row to the top of the original DataFrame
df_with_column_names = pd.concat([column_names_row, df]).reset_index(drop=True)
df_with_column_names.columns = ['time', 'am/pm', 'temp', 'dew point', 'humidity', 'wind speed']

# Display the resulting DataFrame
df_with_column_names.head()









In [19]:
#3.15 getting all data from dates:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
import os
from datetime import datetime, timedelta
import time

# Function to create a folder if it doesn't exist
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (without a graphical user interface)
chrome_service = ChromeService(executable_path='/Users/samdvorin/Desktop/code/542/weather-prediction/chromedriver')  # Replace with the path to your chromedriver executable

# Specify the folder name to save all text files
output_folder = "austin_weather_data"
create_folder(output_folder)

# Loop through dates from February 1 to March 15
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 3, 16)  # March 15 + 1 day
current_date = start_date

while current_date < end_date:
    current_date_str = current_date.strftime('%Y-%m-%d')
    print(f"Scraping data for {current_date_str}...")
    start_time = time.time()

    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

    #url = f'https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/{current_date_str}'
    url = f'https://www.wunderground.com/history/daily/us/tx/austin/KAUS/date/{current_date_str}'
    driver.get(url)

    # Locate the table element at the bottom of the page using your specified XPath
    table_element = driver.find_element('xpath', '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]')

    # Extract the text from the table
    table_text = table_element.text

    # Save the text to a file in the output folder
    with open(os.path.join(output_folder, f"{current_date.strftime('%Y-%m-%d')}.txt"), 'w') as file:
        file.write(table_text)

    driver.quit()

    end_time = time.time()
    duration = end_time - start_time
    print(f"Data for {current_date_str} scraped and saved in {duration:.2f} seconds")

    # Move to the next date
    current_date += timedelta(days=1)





#ERROR ON FEB 18 loook into it!!!!

Scraping data for 2024-02-01...
Data for 2024-02-01 scraped and saved in 88.26 seconds
Scraping data for 2024-02-02...


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]"}
  (Session info: chrome-headless-shell=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102d5453c chromedriver + 3966268
1   chromedriver                        0x0000000102d4cac8 chromedriver + 3934920
2   chromedriver                        0x00000001029cfda0 chromedriver + 277920
3   chromedriver                        0x0000000102a12394 chromedriver + 549780
4   chromedriver                        0x0000000102a4abf0 chromedriver + 781296
5   chromedriver                        0x0000000102a06fb0 chromedriver + 5

In [ ]:

# weird bug on 3.15 returns same file for all tables

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
import os
from datetime import datetime, timedelta
import time

#URLS 
'''
url = f'https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/{current_date_str}'

https://www.wunderground.com/history/daily/us/tx/austin/KAUS/date/2024-3-15

https://www.wunderground.com/history/daily/us/il/chicago/KMDW/date/2024-3-15

https://www.wunderground.com/history/daily/us/fl/miami/KMIA/date/2024-3-15


'''

# Function to create a folder if it doesn't exist
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (without a graphical user interface)
chrome_service = ChromeService(executable_path='/Users/samdvorin/Desktop/code/542/weather-prediction/chromedriver')  # Replace with the path to your chromedriver executable

# Specify the folder name to save all text files
output_folder = "austin_weather_data"
create_folder(output_folder)

# Loop through dates from February 1 to March 15
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 3, 16)  # March 15 + 1 day
current_date = start_date

while current_date < end_date:
    current_date_str = current_date.strftime('%Y-%m-%d')
    print(f"Scraping data for {current_date_str}...")
    start_time = time.time()

    try:
        driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

        url = f'https://www.wunderground.com/history/daily/us/tx/austin/KAUS/date/{current_date_str}'
        driver.get(url)

        # Locate the table element at the bottom of the page using your specified XPath
        table_element = driver.find_element('xpath', '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]')

        # Extract the text from the table
        table_text = table_element.text

        # Save the text to a file in the output folder
        with open(os.path.join(output_folder, f"{current_date.strftime('%Y-%m-%d')}.txt"), 'w') as file:
            file.write(table_text)

    except Exception as e:
        print(f"An error occurred for {current_date_str}: {e}")
    finally:
        if 'driver' in locals():
            driver.quit()

    end_time = time.time()
    duration = end_time - start_time
    print(f"Data for {current_date_str} scraped and saved in {duration:.2f} seconds")

    # Move to the next date
    current_date += timedelta(days=1)


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
import os
from datetime import datetime, timedelta
import time

# Function to create a folder if it doesn't exist
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (without a graphical user interface)
chrome_service = ChromeService(executable_path='/Users/samdvorin/Desktop/code/542/weather-prediction/chromedriver')  # Replace with the path to your chromedriver executable

# Create a single WebDriver instance
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Specify the folder name to save all text files
output_folder = "miami_weather_data"
create_folder(output_folder)

# Loop through dates from February 1 to March 15
start_date = datetime(2024, 2, 5)
end_date = datetime(2024, 3, 16)  # March 15 + 1 day
current_date = start_date

try:
    while current_date < end_date:
        current_date_str = current_date.strftime('%Y-%m-%d')
        print(f"Scraping data for {current_date_str}...")
        start_time = time.time()

        try:
            url = f'https://www.wunderground.com/history/daily/us/fl/miami/KMIA/date/{current_date_str}'
            driver.get(url)

            # Wait for a maximum of 250 seconds for the page to load
            driver.set_page_load_timeout(250)

            # Locate the table element at the bottom of the page using your specified XPath
            table_element = driver.find_element('xpath', '/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]')

            # Extract the text from the table
            table_text = table_element.text

            # Save the text to a file in the output folder
            with open(os.path.join(output_folder, f"{current_date.strftime('%Y-%m-%d')}.txt"), 'w') as file:
                file.write(table_text)

        except Exception as e:
            print(f"An error occurred for {current_date_str}: {e}")

        finally:
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {current_date_str} scraped and saved in {duration:.2f} seconds")

            # Move to the next date
            current_date += timedelta(days=1)

finally:
    # Quit the WebDriver instance after finishing all iterations
    driver.quit()


Scraping data for 2024-02-05...
Data for 2024-02-05 scraped and saved in 55.07 seconds


KeyboardInterrupt: 

In [ ]:
# train model to predict temp line based on the input so far, so train it to predict next temp values at next time step based on the past rows
# basically train it to find next row

